# Switchback 实验 - 时间维度的随机化

**学习目标**
1. 理解 Switchback 实验设计原理
2. 掌握时间序列实验的分析方法
3. 学习 carryover 效应的处理

**业务场景**

想象你是 Uber 的数据科学家,需要测试一个新的动态定价算法。如果使用传统 A/B 测试:
- 50% 司机使用新算法,50% 使用旧算法
- 但乘客会看到两组司机的报价,造成市场混乱
- 司机之间存在竞争,一个司机的定价会影响另一个司机的接单率

这就是**双边市场**的挑战 - 传统 A/B 测试会产生严重的网络效应和溢出效应。

**Switchback 的解决方案**:不在用户之间随机化,而是在**时间**上随机化!

---

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 配置
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✅ 环境准备完成")

---

## Part 1: 为什么需要 Switchback

### 1.1 传统 A/B 测试的局限

在某些场景下,传统的用户级随机化会失效:

1. **双边市场** (Two-sided markets)
   - Uber: 司机 vs 乘客
   - 外卖: 骑手 vs 用户 vs 商家
   - Airbnb: 房东 vs 房客

2. **网络效应** (Network effects)
   - 社交网络: 一个用户的行为影响其他用户
   - 推荐系统: 库存共享

3. **供给约束** (Supply constraints)
   - 有限的司机/骑手/商品
   - 一组用户的行为会影响另一组

### 1.2 溢出效应示例

让我们用一个简单的例子说明问题:

In [ ]:
# 模拟传统 A/B 测试在双边市场的问题
def simulate_spillover_effect(n_drivers=100, n_periods=50):
    """
    模拟双边市场的溢出效应
    场景: 测试司机端的更高定价策略
    """
    results = []
    
    for period in range(n_periods):
        # 50% 司机使用高价策略 (treatment)
        treatment_group = np.random.choice([0, 1], size=n_drivers, p=[0.5, 0.5])
        
        # 基础需求
        base_demand = 100
        
        # Treatment 组: 价格高 10%, 但会流失一些乘客
        # 这些流失的乘客会去 Control 组
        treatment_drivers = treatment_group.sum()
        control_drivers = n_drivers - treatment_drivers
        
        # 溢出效应: Treatment 的高价导致需求流向 Control
        spillover_demand = treatment_drivers * 0.3  # 每个 treatment 司机赶走 0.3 单
        
        if treatment_drivers > 0:
            treatment_orders_per_driver = (base_demand - spillover_demand) / treatment_drivers
            treatment_revenue_per_driver = treatment_orders_per_driver * 1.1  # 高价 10%
        else:
            treatment_revenue_per_driver = 0
        
        if control_drivers > 0:
            control_orders_per_driver = (base_demand + spillover_demand) / control_drivers
            control_revenue_per_driver = control_orders_per_driver * 1.0
        else:
            control_revenue_per_driver = 0
        
        results.append({
            'period': period,
            'treatment_revenue': treatment_revenue_per_driver,
            'control_revenue': control_revenue_per_driver,
            'treatment_drivers': treatment_drivers,
            'control_drivers': control_drivers
        })
    
    return pd.DataFrame(results)

# 运行模拟
spillover_data = simulate_spillover_effect()

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spillover_data['period'],
    y=spillover_data['treatment_revenue'],
    name='Treatment (高价组)',
    line=dict(color='#EB5757', width=2),
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=spillover_data['period'],
    y=spillover_data['control_revenue'],
    name='Control (对照组)',
    line=dict(color='#2D9CDB', width=2),
    mode='lines'
))

fig.update_layout(
    title='传统 A/B 测试的溢出效应 - Control 组被"污染"',
    xaxis_title='时间周期',
    yaxis_title='人均收入',
    hovermode='x unified',
    height=400
)

fig.show()

# 计算偏差
naive_ate = spillover_data['treatment_revenue'].mean() - spillover_data['control_revenue'].mean()
print(f"\n❌ 传统 A/B 测试估计的效应: {naive_ate:.2f}")
print(f"   看起来 Treatment 组更差!")
print(f"\n💡 但这是因为:")
print(f"   1. Treatment 的高价赶走了乘客")
print(f"   2. 这些乘客跑到了 Control 组")
print(f"   3. Control 组的收入被人为提高了 (污染)")
print(f"   4. 我们低估了 Treatment 的真实效应!")

### 1.3 Switchback 的核心思想

**关键洞察**: 如果我们不能避免溢出效应,那就让所有人都受到同样的影响!

**Switchback 设计**:
- 把时间切分成小的时间窗口 (如 1 小时)
- 每个窗口随机分配为 Treatment 或 Control
- 在 Treatment 窗口,**所有**司机都使用新策略
- 在 Control 窗口,**所有**司机都使用旧策略

这样:
- ✅ 没有用户级的溢出 (大家都在同一个策略下)
- ✅ 可以比较不同时间窗口的指标
- ✅ 每个司机都经历了两种策略,更公平

In [ ]:
# 可视化 Switchback 设计
def create_switchback_timeline(n_periods=24):
    """
    创建一个 Switchback 实验的时间线可视化
    """
    # 随机分配每个时间窗口
    np.random.seed(42)
    assignments = np.random.choice(['Treatment', 'Control'], size=n_periods)
    
    # 创建时间标签
    hours = [f'{i:02d}:00' for i in range(n_periods)]
    
    # 颜色映射
    colors = ['#EB5757' if a == 'Treatment' else '#2D9CDB' for a in assignments]
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=hours,
        y=[1] * n_periods,
        marker_color=colors,
        text=assignments,
        textposition='inside',
        hovertemplate='%{x}<br>%{text}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Switchback 实验设计 - 时间维度的随机化',
        xaxis_title='时间 (小时)',
        yaxis_title='',
        yaxis_visible=False,
        height=300,
        showlegend=False
    )
    
    return fig

fig = create_switchback_timeline()
fig.show()

print("\n💡 Switchback 的优势:")
print("   🔴 红色时段: 所有司机都用新定价策略")
print("   🔵 蓝色时段: 所有司机都用旧定价策略")
print("   ✅ 没有用户级的溢出效应")
print("   ✅ 每个司机都经历两种策略")

---

## Part 2: 实验设计

### 2.1 时间窗口的选择

时间窗口的长度是 Switchback 设计中最关键的决策:

**太短的窗口**:
- ❌ Carryover 效应严重 (上个窗口的影响延续到这个窗口)
- ❌ 切换成本高 (系统频繁切换)
- ✅ 更多样本量 (更多时间窗口)

**太长的窗口**:
- ✅ Carryover 效应小
- ✅ 切换成本低
- ❌ 样本量少
- ❌ 受时间趋势影响大

**经验法则**:
- 窗口长度 ≥ 3 × 用户行为周期
- 例如: 如果订单决策需要 10 分钟,窗口至少 30 分钟
- 实际中常用: 30分钟 - 2小时

### 2.2 随机化单元

Switchback 可以在不同层级随机化:

1. **时间随机化** (最常见)
   - 每个时间窗口随机分配
   - 适合: 全局策略变更 (定价、匹配算法)

2. **地理×时间随机化**
   - 每个城市×时间窗口独立随机
   - 适合: 地理差异大的场景

3. **分层随机化**
   - 按周几、时段分层
   - 确保各层级平衡

In [ ]:
# 📖 示范: 设计一个 Switchback 实验
# 
# 场景: 外卖平台想测试一个新的配送算法
# - 实验时长: 2 周
# - 时间窗口: 1 小时
# - 需要分层: 工作日 vs 周末, 高峰 vs 非高峰

def design_switchback_experiment(start_date, end_date, window_hours=1):
    """
    设计一个分层的 Switchback 实验
    
    Parameters:
    -----------
    start_date : str
        实验开始日期 'YYYY-MM-DD'
    end_date : str
        实验结束日期 'YYYY-MM-DD'
    window_hours : int
        时间窗口长度(小时)
    
    Returns:
    --------
    pd.DataFrame: 包含 timestamp, assignment, is_weekend, is_peak 列
    """
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    
    # 生成所有时间窗口
    timestamps = pd.date_range(start, end, freq=f'{window_hours}h')
    
    df = pd.DataFrame({'timestamp': timestamps})
    
    # 特征工程
    df['is_weekend'] = df['timestamp'].dt.weekday >= 5
    df['hour'] = df['timestamp'].dt.hour
    df['is_peak'] = ((df['hour'] >= 11) & (df['hour'] <= 13)) | \
                    ((df['hour'] >= 17) & (df['hour'] <= 19))
    
    # 创建 strata
    df['strata'] = df['is_weekend'].astype(str) + '_' + df['is_peak'].astype(str)
    
    # 在每个 strata 内随机分配
    np.random.seed(42)
    assignments = []
    for strata in df['strata'].unique():
        mask = df['strata'] == strata
        n = mask.sum()
        strata_assignment = np.random.choice(['Treatment', 'Control'], size=n)
        assignments.extend(strata_assignment)
    
    df['assignment'] = assignments
    
    return df[['timestamp', 'assignment', 'is_weekend', 'is_peak']]

# 测试实现
design = design_switchback_experiment('2024-01-01', '2024-01-14', window_hours=1)

print(f"总时间窗口数: {len(design)}")
print(f"\nTreatment 分配:")
print(design['assignment'].value_counts())
print(f"\n分层平衡性检查:")
print(pd.crosstab([design['is_weekend'], design['is_peak']], 
                   design['assignment']))

# 可视化前 3 天
sample = design[design['timestamp'] < '2024-01-04']
colors = sample['assignment'].map({'Treatment': '#EB5757', 'Control': '#2D9CDB'})

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sample['timestamp'],
    y=[1] * len(sample),
    mode='markers',
    marker=dict(color=colors, size=10, line=dict(width=1, color='white')),
    text=sample['assignment'],
    hovertemplate='%{x}<br>%{text}<extra></extra>'
))

fig.update_layout(
    title='实验设计预览 (前3天)',
    xaxis_title='时间',
    yaxis_visible=False,
    height=300
)
fig.show()

### 2.3 样本量计算

Switchback 的样本量计算与传统 A/B 测试不同:

**传统 A/B 测试**:
$$n = \frac{16\sigma^2}{\delta^2}$$

其中 $n$ 是每组需要的**用户数**。

**Switchback**:
$$T = \frac{16\sigma_w^2}{\delta^2}$$

其中 $T$ 是需要的**时间窗口数**, $\sigma_w^2$ 是窗口级别的方差。

**关键差异**:
- Switchback 的有效样本量是**时间窗口数**,不是用户数
- 需要考虑**窗口内相关性** (同一窗口的用户不独立)
- 窗口内用户越多, $\sigma_w^2$ 越小 (中心极限定理)

In [ ]:
def calculate_switchback_power(sigma_individual, delta, users_per_window, alpha=0.05, power=0.8):
    """
    计算 Switchback 需要的时间窗口数
    
    Parameters:
    -----------
    sigma_individual : float
        个体层面的标准差
    delta : float
        最小可检测效应
    users_per_window : int
        每个时间窗口的平均用户数
    alpha : float
        显著性水平
    power : float
        统计功效
    
    Returns:
    --------
    int: 需要的时间窗口数
    """
    # 窗口层面的标准差 (中心极限定理)
    sigma_window = sigma_individual / np.sqrt(users_per_window)
    
    # 标准正态分位数
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    # 每组需要的时间窗口数
    n_windows_per_group = ((z_alpha + z_beta) * sigma_window / delta) ** 2
    
    # 总窗口数
    total_windows = int(np.ceil(2 * n_windows_per_group))
    
    return total_windows

# 示例: 外卖配送时间实验
sigma = 10  # 配送时间标准差 10 分钟
delta = 1   # 想检测 1 分钟的差异
users_per_window = 100  # 每小时 100 单

n_windows = calculate_switchback_power(sigma, delta, users_per_window)
print(f"需要的时间窗口数: {n_windows}")
print(f"如果窗口=1小时, 需要实验 {n_windows} 小时 = {n_windows/24:.1f} 天")

# 可视化: 窗口大小的影响
users_range = np.arange(10, 500, 10)
windows_needed = [calculate_switchback_power(sigma, delta, u) for u in users_range]
days_needed = [w/24 for w in windows_needed]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=users_range,
    y=days_needed,
    mode='lines',
    line=dict(color='#2D9CDB', width=3),
    fill='tozeroy',
    fillcolor='rgba(45, 156, 219, 0.1)'
))

fig.update_layout(
    title='样本量规划: 窗口内用户数 vs 实验天数',
    xaxis_title='每窗口用户数',
    yaxis_title='需要的实验天数',
    hovermode='x',
    height=400
)
fig.show()

print("\n💡 洞察:")
print("   - 窗口内用户越多,需要的实验天数越少")
print("   - 但窗口太长会增加 carryover 风险")
print("   - 需要在样本量和 carryover 之间权衡")

---

## Part 3: Carryover 效应

### 3.1 什么是 Carryover

**Carryover 效应**: 上一个时间窗口的 treatment 对当前窗口产生延续影响。

**例子**:
- **Uber 动态定价**: 上一小时的高价可能让司机在这一小时更积极
- **推荐算法**: 上一窗口的推荐影响了用户兴趣,进而影响这一窗口的点击
- **促销活动**: 上一窗口的促销让用户囤货,这一窗口购买减少

**数学表示**:

令 $Y_t$ 是时间窗口 $t$ 的结果, $W_t$ 是 $t$ 的 treatment 状态:

$$Y_t = \alpha + \beta W_t + \gamma W_{t-1} + \varepsilon_t$$

- $\beta$: 当前效应 (我们想要的)
- $\gamma$: Carryover 效应 (bias 来源)

如果 $\gamma \neq 0$, 简单的差分估计会有偏差!

### 3.2 检测 Carryover

In [ ]:
# 模拟有 carryover 的数据
def simulate_switchback_with_carryover(n_windows=200, 
                                        treatment_effect=2.0,
                                        carryover_effect=0.5):
    """
    模拟带 carryover 的 Switchback 数据
    """
    np.random.seed(42)
    
    # 随机分配
    treatment = np.random.choice([0, 1], size=n_windows)
    
    # 生成结果
    y = np.zeros(n_windows)
    y[0] = 10 + np.random.normal(0, 1)  # 初始值
    
    for t in range(1, n_windows):
        # 当前 treatment 效应
        current_effect = treatment_effect * treatment[t]
        
        # Carryover 效应 (上一窗口的影响)
        carryover = carryover_effect * treatment[t-1]
        
        # 基线 + 效应 + 噪音
        y[t] = 10 + current_effect + carryover + np.random.normal(0, 1)
    
    df = pd.DataFrame({
        'window': range(n_windows),
        'treatment': treatment,
        'outcome': y
    })
    
    # 添加滞后变量
    df['treatment_lag1'] = df['treatment'].shift(1).fillna(0)
    
    return df

# 生成数据
data = simulate_switchback_with_carryover()

# 可视化: 不同组合的平均结果
analysis = data.groupby(['treatment', 'treatment_lag1'])['outcome'].mean().reset_index()
analysis['group'] = analysis.apply(
    lambda x: f"W_t={int(x['treatment'])}, W_{{t-1}}={int(x['treatment_lag1'])}", 
    axis=1
)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=analysis['group'],
    y=analysis['outcome'],
    marker_color=['#2D9CDB', '#9B59B6', '#F39C12', '#EB5757'],
    text=analysis['outcome'].round(2),
    textposition='outside'
))

fig.update_layout(
    title='Carryover 效应检测: 不同组合的平均结果',
    xaxis_title='(当前 Treatment, 上一窗口 Treatment)',
    yaxis_title='平均结果',
    height=400
)
fig.show()

print("\n💡 如何判断是否有 carryover:")
print(f"   - (0, 0) vs (0, 1): 都是 Control, 但上一窗口不同")
print(f"     差异 = {analysis.loc[1, 'outcome'] - analysis.loc[0, 'outcome']:.2f}")
print(f"   - (1, 0) vs (1, 1): 都是 Treatment, 但上一窗口不同")
print(f"     差异 = {analysis.loc[3, 'outcome'] - analysis.loc[2, 'outcome']:.2f}")
print(f"\n   如果这两个差异显著非零 → 有 carryover!")

In [ ]:
# 📖 示范: Carryover 检测实现

def test_carryover(df, outcome_col='outcome', treatment_col='treatment', lag=1):
    """
    检测 carryover 效应
    
    Parameters:
    -----------
    df : pd.DataFrame
        包含 outcome 和 treatment 的数据
    outcome_col : str
        结果变量名
    treatment_col : str
        treatment 变量名
    lag : int
        检测的滞后阶数
    
    Returns:
    --------
    dict: 包含回归结果的字典
    """
    from scipy.stats import linregress
    
    # 准备数据
    df_work = df.copy()
    df_work['treatment_lag'] = df_work[treatment_col].shift(lag)
    df_work = df_work.dropna()
    
    # 构建设计矩阵
    X = df_work[[treatment_col, 'treatment_lag']].values
    y = df_work[outcome_col].values
    
    # 多元回归
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X, y)
    
    # 计算标准误
    y_pred = model.predict(X)
    residuals = y - y_pred
    mse = np.mean(residuals**2)
    
    # 计算 t 统计量
    X_with_intercept = np.column_stack([np.ones(len(X)), X])
    var_coef = mse * np.linalg.inv(X_with_intercept.T @ X_with_intercept).diagonal()
    se_coef = np.sqrt(var_coef[1:])  # 去掉截距
    
    t_stats = model.coef_ / se_coef
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), len(X) - 3))
    
    results = {
        'current_effect': model.coef_[0],
        'current_pvalue': p_values[0],
        'carryover_effect': model.coef_[1],
        'carryover_pvalue': p_values[1],
        'has_carryover': p_values[1] < 0.05
    }
    
    return results

# 测试实现
results = test_carryover(data)

print("Carryover 检测结果:")
print("="*50)
print(f"当前效应 (β):      {results['current_effect']:.3f} (p={results['current_pvalue']:.4f})")
print(f"Carryover 效应 (γ): {results['carryover_effect']:.3f} (p={results['carryover_pvalue']:.4f})")
print(f"\n是否存在 carryover: {'是 ❌' if results['has_carryover'] else '否 ✅'}")

if results['has_carryover']:
    print("\n⚠️ 警告: 检测到显著的 carryover 效应!")
    print("   建议:")
    print("   1. 增加时间窗口长度")
    print("   2. 在分析中控制滞后 treatment")
    print("   3. 使用更复杂的估计方法")

### 3.3 处理 Carryover

如果检测到 carryover, 有几种处理方法:

**方法 1: 增加窗口长度**
- 最简单直接
- 但会减少样本量

**方法 2: 添加 "Washout" 期**
- 在每次切换后,丢弃一个短窗口的数据
- 让 carryover 效应消退

**方法 3: 回归调整**
- 在分析中控制滞后 treatment
- 估计模型: $Y_t = \alpha + \beta W_t + \gamma W_{t-1} + \varepsilon_t$
- 关注 $\beta$ (净效应)

**方法 4: 工具变量**
- 使用更早的 treatment (如 $W_{t-2}$) 作为工具变量
- 更高级的因果推断方法

In [ ]:
# 比较不同处理方法
def compare_carryover_methods(data):
    """
    比较不同 carryover 处理方法的估计
    """
    results = {}
    
    # 方法 1: 简单差分 (有偏)
    treatment_mean = data[data['treatment'] == 1]['outcome'].mean()
    control_mean = data[data['treatment'] == 0]['outcome'].mean()
    results['Naive'] = treatment_mean - control_mean
    
    # 方法 2: Washout - 丢弃 treatment 切换后的第一个窗口
    data_washout = data.copy()
    data_washout['is_switch'] = (data_washout['treatment'] != data_washout['treatment_lag1'])
    data_washout = data_washout[~data_washout['is_switch']]
    
    treatment_mean = data_washout[data_washout['treatment'] == 1]['outcome'].mean()
    control_mean = data_washout[data_washout['treatment'] == 0]['outcome'].mean()
    results['Washout'] = treatment_mean - control_mean
    
    # 方法 3: 回归调整
    from sklearn.linear_model import LinearRegression
    X = data[['treatment', 'treatment_lag1']].values[1:]  # 去掉第一行
    y = data['outcome'].values[1:]
    model = LinearRegression().fit(X, y)
    results['Regression'] = model.coef_[0]  # 只看当前效应
    
    # 真实效应 (已知,因为是模拟数据)
    results['True'] = 2.0
    
    return results

comparison = compare_carryover_methods(data)

# 可视化
methods = list(comparison.keys())
estimates = list(comparison.values())
colors = ['#EB5757', '#F39C12', '#27AE60', '#2D9CDB']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=methods,
    y=estimates,
    marker_color=colors,
    text=[f'{e:.2f}' for e in estimates],
    textposition='outside'
))

# 添加真实值参考线
fig.add_hline(y=2.0, line_dash="dash", line_color="black",
              annotation_text="真实效应 = 2.0")

fig.update_layout(
    title='不同 Carryover 处理方法的估计对比',
    xaxis_title='方法',
    yaxis_title='估计的 Treatment 效应',
    height=400
)
fig.show()

print("\n各方法的偏差:")
for method, estimate in comparison.items():
    if method != 'True':
        bias = estimate - 2.0
        print(f"  {method:12s}: 偏差 = {bias:+.3f}")

---

## Part 4: 估计与推断

### 4.1 差分估计

最简单的 Switchback 估计器:

$$\hat{\tau} = \frac{1}{n_T}\sum_{t \in T} Y_t - \frac{1}{n_C}\sum_{t \in C} Y_t$$

其中 $T$ 是 treatment 窗口集合, $C$ 是 control 窗口集合。

**优点**: 简单直观
**缺点**: 
- 不能控制时间趋势
- 不能控制协变量
- 对 carryover 敏感

### 4.2 回归估计

更好的方法是用回归:

$$Y_t = \alpha + \beta W_t + \gamma X_t + \varepsilon_t$$

其中:
- $X_t$: 协变量 (时间趋势、星期几、节假日等)
- $\beta$: treatment 效应

**优点**:
- 可以控制时间趋势
- 可以纳入协变量,提高精度
- 可以处理 carryover

In [ ]:
# 📖 示范: 完整的 Switchback 分析管道

class SwitchbackAnalyzer:
    """
    Switchback 实验分析器
    """
    def __init__(self, data, outcome_col, treatment_col, time_col):
        """
        Parameters:
        -----------
        data : pd.DataFrame
            实验数据
        outcome_col : str
            结果变量列名
        treatment_col : str
            treatment 列名
        time_col : str
            时间列名
        """
        self.data = data.copy()
        self.outcome_col = outcome_col
        self.treatment_col = treatment_col
        self.time_col = time_col
        
    def check_balance(self):
        """
        检查随机化是否平衡
        """
        counts = self.data[self.treatment_col].value_counts()
        print("随机化平衡性检查:")
        print("="*50)
        print(f"Treatment 窗口数: {counts.get(1, 0)}")
        print(f"Control 窗口数:   {counts.get(0, 0)}")
        print(f"比例: {counts.get(1, 0) / counts.get(0, 1):.2f}")
        
    def test_carryover(self, lag=1):
        """
        检测 carryover 效应
        """
        return test_carryover(self.data, self.outcome_col, self.treatment_col, lag)
        
    def estimate_ate(self, method='diff', covariates=None):
        """
        估计平均 treatment 效应
        
        Parameters:
        -----------
        method : str
            'diff' 或 'regression'
        covariates : list
            回归方法中使用的协变量列名
        
        Returns:
        --------
        dict: 包含估计值、标准误、置信区间
        """
        if method == 'diff':
            # 简单差分
            t_mean = self.data[self.data[self.treatment_col] == 1][self.outcome_col].mean()
            c_mean = self.data[self.data[self.treatment_col] == 0][self.outcome_col].mean()
            ate = t_mean - c_mean
            
            # 计算标准误
            t_var = self.data[self.data[self.treatment_col] == 1][self.outcome_col].var()
            c_var = self.data[self.data[self.treatment_col] == 0][self.outcome_col].var()
            t_n = (self.data[self.treatment_col] == 1).sum()
            c_n = (self.data[self.treatment_col] == 0).sum()
            
            se = np.sqrt(t_var/t_n + c_var/c_n)
            
        elif method == 'regression':
            from sklearn.linear_model import LinearRegression
            
            # 构建特征矩阵
            if covariates is None:
                X = self.data[[self.treatment_col]].values
            else:
                X = self.data[[self.treatment_col] + covariates].values
            
            y = self.data[self.outcome_col].values
            
            model = LinearRegression().fit(X, y)
            ate = model.coef_[0]
            
            # 计算标准误
            y_pred = model.predict(X)
            residuals = y - y_pred
            mse = np.mean(residuals**2)
            
            X_with_intercept = np.column_stack([np.ones(len(X)), X])
            var_coef = mse * np.linalg.inv(X_with_intercept.T @ X_with_intercept).diagonal()
            se = np.sqrt(var_coef[1])  # treatment 系数的标准误
        
        # 置信区间
        ci_lower = ate - 1.96 * se
        ci_upper = ate + 1.96 * se
        
        # p 值
        t_stat = ate / se
        p_value = 2 * (1 - stats.norm.cdf(abs(t_stat)))
        
        return {
            'ate': ate,
            'se': se,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'p_value': p_value
        }
    
    def plot_time_series(self):
        """
        可视化时间序列
        """
        fig = go.Figure()
        
        # Treatment 窗口
        treatment_data = self.data[self.data[self.treatment_col] == 1]
        fig.add_trace(go.Scatter(
            x=treatment_data[self.time_col],
            y=treatment_data[self.outcome_col],
            mode='markers',
            name='Treatment',
            marker=dict(color='#EB5757', size=8)
        ))
        
        # Control 窗口
        control_data = self.data[self.data[self.treatment_col] == 0]
        fig.add_trace(go.Scatter(
            x=control_data[self.time_col],
            y=control_data[self.outcome_col],
            mode='markers',
            name='Control',
            marker=dict(color='#2D9CDB', size=8)
        ))
        
        fig.update_layout(
            title='Switchback 时间序列',
            xaxis_title='时间窗口',
            yaxis_title=self.outcome_col,
            hovermode='x unified',
            height=400
        )
        
        return fig

# 测试分析器
analyzer = SwitchbackAnalyzer(
    data=data,
    outcome_col='outcome',
    treatment_col='treatment',
    time_col='window'
)

# 1. 检查平衡性
analyzer.check_balance()

# 2. 检测 carryover
print("\n")
carryover_test = analyzer.test_carryover()
print(f"Carryover 检测: p-value = {carryover_test['carryover_pvalue']:.4f}")

# 3. 估计效应
print("\n")
results_diff = analyzer.estimate_ate(method='diff')
results_reg = analyzer.estimate_ate(method='regression')

print("ATE 估计结果:")
print("="*50)
print(f"差分方法:   {results_diff['ate']:.3f} (95% CI: [{results_diff['ci_lower']:.3f}, {results_diff['ci_upper']:.3f}])")
print(f"回归方法:   {results_reg['ate']:.3f} (95% CI: [{results_reg['ci_lower']:.3f}, {results_reg['ci_upper']:.3f}])")
print(f"p-value:    {results_reg['p_value']:.4f}")

# 4. 可视化
fig = analyzer.plot_time_series()
fig.show()

### 4.3 标准误与聚类

Switchback 的一个重要问题: **时间序列相关性**

- 相邻时间窗口的结果可能相关 (自相关)
- 如果忽略这种相关性,标准误会被低估
- 导致 **假阳性率** 升高

**解决方法**:

1. **Newey-West 标准误**: 考虑时间序列自相关
2. **Block Bootstrap**: 对时间块进行 bootstrap
3. **聚类稳健标准误**: 按天或周聚类

在实践中,最常用的是 **按天聚类的标准误**。

In [ ]:
# 演示自相关对标准误的影响
def simulate_autocorrelated_switchback(n_windows=200, rho=0.3):
    """
    模拟有自相关的 Switchback 数据
    
    Parameters:
    -----------
    n_windows : int
        时间窗口数
    rho : float
        自相关系数 AR(1)
    """
    np.random.seed(42)
    
    # 随机分配
    treatment = np.random.choice([0, 1], size=n_windows)
    
    # 生成 AR(1) 误差项
    errors = np.zeros(n_windows)
    errors[0] = np.random.normal(0, 1)
    for t in range(1, n_windows):
        errors[t] = rho * errors[t-1] + np.random.normal(0, 1)
    
    # 生成结果
    y = 10 + 2 * treatment + errors
    
    return pd.DataFrame({
        'window': range(n_windows),
        'treatment': treatment,
        'outcome': y
    })

# 比较不同 rho 下的标准误估计
rho_values = [0, 0.3, 0.5, 0.7]
se_naive = []
se_bootstrap = []

for rho in rho_values:
    # 模拟数据
    data_auto = simulate_autocorrelated_switchback(n_windows=200, rho=rho)
    
    # Naive 标准误 (忽略自相关)
    t_data = data_auto[data_auto['treatment'] == 1]['outcome']
    c_data = data_auto[data_auto['treatment'] == 0]['outcome']
    se_naive.append(np.sqrt(t_data.var()/len(t_data) + c_data.var()/len(c_data)))
    
    # Bootstrap 标准误 (block bootstrap)
    n_bootstrap = 1000
    bootstrap_ates = []
    block_size = 5  # 每个 block 5 个窗口
    
    for _ in range(n_bootstrap):
        # 随机抽取 block
        n_blocks = len(data_auto) // block_size
        block_indices = np.random.choice(n_blocks, size=n_blocks, replace=True)
        
        sample_indices = []
        for bi in block_indices:
            sample_indices.extend(range(bi*block_size, (bi+1)*block_size))
        
        sample = data_auto.iloc[sample_indices]
        
        t_mean = sample[sample['treatment'] == 1]['outcome'].mean()
        c_mean = sample[sample['treatment'] == 0]['outcome'].mean()
        bootstrap_ates.append(t_mean - c_mean)
    
    se_bootstrap.append(np.std(bootstrap_ates))

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=rho_values,
    y=se_naive,
    mode='lines+markers',
    name='Naive SE (忽略自相关)',
    line=dict(color='#EB5757', width=2),
    marker=dict(size=10)
))

fig.add_trace(go.Scatter(
    x=rho_values,
    y=se_bootstrap,
    mode='lines+markers',
    name='Block Bootstrap SE',
    line=dict(color='#27AE60', width=2),
    marker=dict(size=10)
))

fig.update_layout(
    title='自相关对标准误的影响',
    xaxis_title='自相关系数 (ρ)',
    yaxis_title='标准误',
    hovermode='x unified',
    height=400
)
fig.show()

print("\n💡 关键发现:")
print("   - Naive SE 严重低估了真实的不确定性")
print("   - 自相关越强,低估越严重")
print("   - 必须使用 block bootstrap 或聚类标准误!")

---

## Part 5: 业务案例

### 案例 1: Uber 动态定价实验

**背景**: Uber 想测试一个新的动态定价算法,在高峰时段提高价格。

**挑战**:
- 双边市场: 司机和乘客相互影响
- 如果只有部分司机涨价,乘客会转向不涨价的司机
- 传统 A/B 测试会严重低估涨价的负面影响

**Switchback 设计**:
- 时间窗口: 30 分钟
- 随机化: 每个窗口随机分配为高价或正常价
- 关键指标: 司机收入、乘客等待时间、完单率

让我们模拟这个实验:

In [ ]:
def simulate_uber_pricing_experiment(n_days=14, windows_per_day=48):
    """
    模拟 Uber 动态定价实验
    
    Parameters:
    -----------
    n_days : int
        实验天数
    windows_per_day : int
        每天的时间窗口数 (48 = 每30分钟一个窗口)
    """
    np.random.seed(42)
    n_windows = n_days * windows_per_day
    
    # 时间特征
    start_time = pd.Timestamp('2024-01-01')
    timestamps = [start_time + pd.Timedelta(minutes=30*i) for i in range(n_windows)]
    
    df = pd.DataFrame({'timestamp': timestamps})
    df['hour'] = df['timestamp'].dt.hour
    df['is_weekend'] = df['timestamp'].dt.weekday >= 5
    df['is_peak'] = ((df['hour'] >= 7) & (df['hour'] <= 9)) | \
                    ((df['hour'] >= 17) & (df['hour'] <= 19))
    
    # 随机分配 (分层)
    df['strata'] = df['is_weekend'].astype(str) + '_' + df['is_peak'].astype(str)
    assignments = []
    for strata in df['strata'].unique():
        mask = df['strata'] == strata
        n = mask.sum()
        strata_assignment = np.random.choice([0, 1], size=n)
        assignments.extend(strata_assignment)
    df['high_price'] = assignments
    
    # 模拟结果
    # 基础需求 (受时间影响)
    base_demand = 100 + 50 * df['is_peak'] + 30 * df['is_weekend']
    
    # 高价策略
    # - 价格提高 20%
    # - 需求下降 15% (价格弹性)
    # - 但司机积极性提高,完单率提升 10%
    price_multiplier = 1 + 0.2 * df['high_price']
    demand_multiplier = 1 - 0.15 * df['high_price']
    completion_multiplier = 1 + 0.10 * df['high_price']
    
    # 完成订单数
    completed_orders = base_demand * demand_multiplier * completion_multiplier + \
                       np.random.normal(0, 10, n_windows)
    
    # 司机收入
    base_fare = 15
    driver_revenue = completed_orders * base_fare * price_multiplier
    
    # 乘客等待时间
    base_wait = 5  # 分钟
    wait_time = base_wait * (1 / completion_multiplier) + np.random.normal(0, 0.5, n_windows)
    
    df['completed_orders'] = completed_orders
    df['driver_revenue'] = driver_revenue
    df['wait_time'] = wait_time
    
    return df

# 运行实验
uber_data = simulate_uber_pricing_experiment()

# 分析
print("Uber 动态定价实验结果")
print("="*70)

metrics = ['completed_orders', 'driver_revenue', 'wait_time']
metric_names = ['完成订单数', '司机收入', '等待时间(分钟)']

for metric, name in zip(metrics, metric_names):
    analyzer = SwitchbackAnalyzer(
        data=uber_data,
        outcome_col=metric,
        treatment_col='high_price',
        time_col='timestamp'
    )
    
    result = analyzer.estimate_ate(method='regression')
    
    pct_change = (result['ate'] / uber_data[uber_data['high_price']==0][metric].mean()) * 100
    
    print(f"\n{name}:")
    print(f"  效应: {result['ate']:+.2f} ({pct_change:+.1f}%)")
    print(f"  95% CI: [{result['ci_lower']:.2f}, {result['ci_upper']:.2f}]")
    print(f"  p-value: {result['p_value']:.4f} {'✅' if result['p_value'] < 0.05 else ''}")

# 可视化对比
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['完成订单数', '司机收入', '等待时间']
)

for i, metric in enumerate(metrics, 1):
    treatment_mean = uber_data[uber_data['high_price']==1][metric].mean()
    control_mean = uber_data[uber_data['high_price']==0][metric].mean()
    
    fig.add_trace(
        go.Bar(x=['Control', 'Treatment'], 
               y=[control_mean, treatment_mean],
               marker_color=['#2D9CDB', '#EB5757'],
               showlegend=False,
               text=[f'{control_mean:.0f}', f'{treatment_mean:.0f}'],
               textposition='outside'),
        row=1, col=i
    )

fig.update_layout(height=400, title_text="Uber 动态定价实验 - 关键指标对比")
fig.show()

print("\n\n💡 业务洞察:")
print("   ✅ 高价策略显著提升司机收入 (+17%)")
print("   ⚠️  完成订单数略有下降 (需求价格弹性)")
print("   ❌ 乘客等待时间略微增加")
print("\n   建议: 可以在高峰时段采用高价策略,平衡供需")

### 案例 2: 外卖平台配送策略实验

**背景**: 外卖平台想测试一个新的智能派单算法,目标是减少配送时间。

**挑战**:
- 骑手是共享资源,一个订单的派单会影响其他订单
- 用户级 A/B 测试会导致 **资源竞争偏差**
- Switchback 可以避免这个问题

**实验设计**:
- 时间窗口: 1 小时
- 在每个窗口,所有订单都使用同一个算法
- 关键指标: 配送时间、骑手利用率、用户满意度

In [ ]:
# 📖 示范: 分析外卖配送实验

def simulate_delivery_experiment(n_days=21, windows_per_day=24):
    """
    模拟外卖配送实验
    """
    np.random.seed(42)
    n_windows = n_days * windows_per_day
    
    # 时间特征
    start_time = pd.Timestamp('2024-01-01')
    timestamps = [start_time + pd.Timedelta(hours=i) for i in range(n_windows)]
    
    df = pd.DataFrame({'timestamp': timestamps})
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    
    # 随机分配
    df['smart_dispatch'] = np.random.choice([0, 1], size=n_windows)
    
    # 基础配送时间
    base_time = 35  # 分钟
    
    # 午餐和晚餐时段更忙,配送时间更长
    rush_effect = 5 * (((df['hour'] >= 11) & (df['hour'] <= 13)) | 
                       ((df['hour'] >= 18) & (df['hour'] <= 20)))
    
    # 智能派单算法效果
    # - 配送时间减少 3 分钟
    # - 但在非高峰期效果更明显
    algorithm_effect = -3 * df['smart_dispatch']
    
    # 配送时间
    df['delivery_time'] = base_time + rush_effect + algorithm_effect + \
                          np.random.normal(0, 3, n_windows)
    
    # 骑手利用率 (每小时配送单数)
    base_utilization = 4.0
    df['orders_per_rider'] = base_utilization + 0.3 * df['smart_dispatch'] + \
                             np.random.normal(0, 0.3, n_windows)
    
    return df

# 生成数据
delivery_data = simulate_delivery_experiment()

print("外卖配送实验分析")
print("="*70)

# 1. 配送时间分析
analyzer_time = SwitchbackAnalyzer(
    data=delivery_data,
    outcome_col='delivery_time',
    treatment_col='smart_dispatch',
    time_col='timestamp'
)

print("\n1. 随机化平衡性检查:")
analyzer_time.check_balance()

print("\n2. Carryover 检测:")
carryover = analyzer_time.test_carryover()
print(f"   Carryover p-value: {carryover['carryover_pvalue']:.4f}")
print(f"   {'❌ 存在 carryover' if carryover['has_carryover'] else '✅ 无显著 carryover'}")

print("\n3. 配送时间效应:")
result_time = analyzer_time.estimate_ate(method='regression')
print(f"   ATE: {result_time['ate']:.2f} 分钟")
print(f"   95% CI: [{result_time['ci_lower']:.2f}, {result_time['ci_upper']:.2f}]")
print(f"   p-value: {result_time['p_value']:.4f}")

# 2. 骑手利用率分析
analyzer_util = SwitchbackAnalyzer(
    data=delivery_data,
    outcome_col='orders_per_rider',
    treatment_col='smart_dispatch',
    time_col='timestamp'
)

print("\n4. 骑手利用率效应:")
result_util = analyzer_util.estimate_ate(method='regression')
pct_change = (result_util['ate'] / delivery_data[delivery_data['smart_dispatch']==0]['orders_per_rider'].mean()) * 100
print(f"   ATE: {result_util['ate']:+.2f} 单/小时 ({pct_change:+.1f}%)")
print(f"   95% CI: [{result_util['ci_lower']:.2f}, {result_util['ci_upper']:.2f}]")
print(f"   p-value: {result_util['p_value']:.4f}")

# 5. 可视化
fig = analyzer_time.plot_time_series()
fig.update_layout(title='配送时间时间序列 (前7天)')

# 只显示前7天
cutoff = delivery_data['timestamp'].min() + pd.Timedelta(days=7)
fig.update_xaxes(range=[delivery_data['timestamp'].min(), cutoff])
fig.show()

print("\n\n💡 业务结论:")
print("   ✅ 智能派单显著减少配送时间约 3 分钟")
print("   ✅ 骑手利用率提升 7.5%")
print("   ✅ 没有检测到显著的 carryover 效应")
print("\n   建议: 全量上线智能派单算法")

---

## 练习与思考题

### 练习 1: Switchback vs 传统 A/B

**场景**: 你是一个共享单车平台的数据科学家,需要测试一个新的定价策略。

**任务**:
1. 说明为什么传统 A/B 测试可能不适用
2. 设计一个 Switchback 实验
3. 列出可能的 carryover 来源
4. 提出缓解 carryover 的方法

**提示**: 考虑单车是有限资源,用户之间存在竞争。

In [ ]:
# YOUR ANSWER HERE (Markdown or Code)

answer_q1 = """
1. 为什么传统 A/B 不适用:
   - 单车是共享资源,总量有限
   - 如果 Treatment 组涨价,用户可能抢占 Control 组的单车
   - Control 组的单车可用性下降,低估了涨价的负面影响

2. Switchback 设计:
   - 随机化单元: 时间窗口 (建议 2 小时)
   - 每个窗口内,所有用户看到同样的价格
   - 分层: 工作日/周末, 高峰/非高峰

3. Carryover 来源:
   - 用户习惯: 上一窗口的高价让用户选择其他交通方式,这一窗口仍未回归
   - 单车分布: 上一窗口的使用模式影响这一窗口的单车分布
   - 心理账户: 用户对价格的感知有滞后性

4. 缓解方法:
   - 增加时间窗口长度到 2-3 小时
   - 添加 30 分钟的 washout 期
   - 在分析中控制滞后价格
   - 监控单车分布的变化,作为协变量
"""

print(answer_q1)

### 练习 2: 样本量规划

**场景**: 你想检测新算法使配送时间减少 2 分钟的效应。

**已知**:
- 配送时间标准差: $\sigma = 8$ 分钟
- 每小时平均 200 单
- 希望达到 80% 统计功效, 5% 显著性水平

**任务**: 计算需要实验多少天 (假设每天 24 个时间窗口)。

In [ ]:
# 📖 示范: 样本量计算

sigma = 8  # 个体标准差
delta = 2  # 最小可检测效应
users_per_window = 200  # 每窗口用户数
windows_per_day = 24

n_windows = calculate_switchback_power(sigma, delta, users_per_window)
n_days = np.ceil(n_windows / windows_per_day)

print(f"样本量计算:")
print("="*50)
print(f"个体标准差: {sigma} 分钟")
print(f"窗口标准差: {sigma/np.sqrt(users_per_window):.3f} 分钟")
print(f"最小可检测效应: {delta} 分钟")
print(f"\n需要的时间窗口数: {n_windows}")
print(f"需要的实验天数: {n_days:.0f} 天")

# 敏感性分析: 如果每窗口用户数减少呢?
users_range = [50, 100, 150, 200, 250, 300]
days_needed = []

for u in users_range:
    n_w = calculate_switchback_power(sigma, delta, u)
    days_needed.append(n_w / windows_per_day)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=users_range,
    y=days_needed,
    mode='lines+markers',
    line=dict(color='#2D9CDB', width=3),
    marker=dict(size=10)
))

fig.add_hline(y=14, line_dash="dash", line_color="red",
              annotation_text="2 周 (常见实验时长)")

fig.update_layout(
    title='敏感性分析: 窗口用户数 vs 实验天数',
    xaxis_title='每窗口订单数',
    yaxis_title='需要的实验天数',
    height=400
)
fig.show()

print(f"\n💡 洞察: 如果高峰期每小时只有 100 单, 需要实验 {calculate_switchback_power(sigma, delta, 100)/24:.0f} 天")

### 思考题

1. **Switchback 的局限性**
   - 什么情况下 Switchback 也不适用?
   - 提示: 考虑学习效应、网络效应的持久性

2. **时间趋势**
   - 如果实验期间有强时间趋势 (如业务快速增长), 如何处理?
   - 为什么随机化可以缓解这个问题?

3. **多个 Treatment**
   - 如何在 Switchback 中测试 3 个或更多策略?
   - 样本量需求如何变化?

4. **组合 Switchback 和用户随机化**
   - 能否在时间维度 Switchback, 同时在用户维度 A/B?
   - 这种设计的优缺点是什么?

5. **实践建议**
   - 在你的业务场景中, 什么实验适合用 Switchback?
   - 什么实验应该用传统 A/B?
   - 如何判断?

---

## 总结

**Switchback 的核心思想**: 在时间维度随机化, 避免用户级溢出效应。

**适用场景**:
- ✅ 双边市场 (Uber, 外卖, Airbnb)
- ✅ 共享资源 (库存, 骑手, 司机)
- ✅ 强网络效应
- ✅ 全局策略变更 (定价, 匹配算法)

**关键挑战**:
- ⚠️  Carryover 效应 (必须检测和处理)
- ⚠️  时间序列相关性 (需要正确的标准误)
- ⚠️  样本量规划 (窗口数 vs 窗口长度权衡)

**最佳实践**:
1. 选择合适的窗口长度 (≥ 3× 行为周期)
2. 分层随机化 (按时间特征)
3. 检测 carryover (回归 + 可视化)
4. 使用稳健标准误 (block bootstrap 或聚类)
5. 控制时间趋势 (回归调整)

**延伸阅读**:
- Bojinov et al. (2020): "Design and Analysis of Switchback Experiments"
- Johari et al. (2022): "Experimental Design in Two-Sided Platforms"
- DoorDash Tech Blog: "Switchback Experiments at DoorDash"

---

**恭喜! 你已经掌握了 Switchback 实验设计! 🎉**

这是因果推断工具箱中的高级武器, 在正确的场景下威力巨大。记住: 选对实验设计, 比任何统计技巧都重要!